Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [22]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
lifetime = ratings.groupby('userId').agg(['max', 'min'])['timestamp']
lifetime['lifetime'] = lifetime[['max']].sub(lifetime['min'], axis=0)
ratings_grouped = ratings.groupby('userId').count().sort_values('rating', ascending=False)
joined = ratings_grouped.join(lifetime, how='left')
joined.head()

,movieId,rating,timestamp,max,min,lifetime
userId,,,,,,
547,2391,2391,2391,1476587644,974777109,501810535
564,1868,1868,1868,976224804,974708446,1516358
624,1735,1735,1735,1476616373,1019123866,457492507
15,1700,1700,1700,1469330735,997937239,471393496
73,1610,1610,1610,1476086345,1255500852,220585493


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [23]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [24]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [25]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [26]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [27]:
joined1 = client_base.merge(rzd, how='left', on='client_id').merge(auto, how='left', on='client_id').merge(air, how='left', on='client_id')
joined1.head(10)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [28]:
client_base2 = client_base.drop(['address'], axis=1) #сомневаюсь что предполагалось что нужно так сделать
joined2 = client_base2.merge(rzd, how='left', on='client_id').merge(auto, how='left', on='client_id').merge(air, how='left', on='client_id')
joined2.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Сначала я бы сгруппировала данные координат по наиболее часто встречающимся для каждого пользователя.
Потом проанализировала по количеству покупок, поскольку может быть наиболее частые координаты, а покупок мало или нет, например если это координаты дома.
Отдельно сравнила бы наибольшие траты из таблицы визитов и покупок с координатами, также может быть что покупок по количеству мало а траты большие.
Координаты могут быть удалены друг от друга, но повторяться с одинаковой частотой (например, работа и магазин) или быть редко повторяющимися но с максимальным размером трат.
Зная координаты можно дополнительно выяснить объект, располагающийся по ним (жилой дом, офисное здание, магазин, больница и т.п.),
а также проложить автомобильный и пешеходный маршрут между наиболее часто посещаемыми координатами.